In [123]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re
from flask import Flask
from flask import jsonify, request
import pymysql
import json

In [140]:
#我要翻頁!!!!!!   by 更改url
def thzucrawler(url,pages=1):
    now = datetime.date.today()
    articles = []
    
    response = rq.get(url,cookies={'over18': '1'})
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    pages_actual_text=soup.select('div.pg a')[-2].text
    pages_actual= int(re.search('\d.*',pages_actual_text).group(0))
    
    url_head=re.search('(http://28th.cc/forum.*-)\d.*.html',dd).group(1)
    
    
    if pages < pages_actual:
        pages_actual=pages
    
    for i in range(0,pages_actual):
        response = rq.get(url,cookies={'over18': '1'})
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'lxml')
        page = soup.select("div#pgt div strong")[0].text
        for s in soup.select("div.mn div#threadlist div.bm_c form#moderate table tbody"):
            if (response.status_code == 200) and (re.search('normalthread.*',s['id'])):
                #requests.get() 的結果是 request.Response 物件, 我們可以先透過該物件的 statu_code 屬性取得 server 回覆的狀態碼
                #(例如 200 表示正常, 404 表示找不到網頁等), 若狀態碼為 200, 代表正常回應
                board=s.select("th em a")[0].text
                name=s.select("th a.s.xst")[0].text
                url_articles= s.select("th a.s.xst")[0]['href']
                date=s.select("td span")[0].text
                if re.search('前',date) or re.search('天',date):
                    print(date)
                    if re.search('小时',date) or re.search('秒',date) or re.search('分钟',date):
                        date=now
                    elif re.search('(.*)天',date).group(1)=='昨':
                        date=now+datetime.timedelta(-1)
                    elif re.search('(.*)天',date).group(1)=='前':
                        date=now+datetime.timedelta(-2)
                    else:
                        delta=re.search('(.*)天',date).group(1)
                        date=now+datetime.timedelta(int(delta))
                articles.append({'page': page,'board': board,'title': name,'date': str(date),'url': "http://28th.cc/"+url_articles})
            url=url_head+str(i+2)+".html"
    return articles

In [141]:
raw_datas=thzucrawler("http://28th.cc/forum-39-1.html",1500)

27 分钟前
半小时前
半小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
4 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
5 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 小时前
6 

5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前
5 天前


半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3

半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3

半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3

...

半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时

半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时

半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时

半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时

半小时前
半小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
1 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
2 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前
3 小时前


In [143]:
import pymysql
import time
# import charts
# 資料庫參數設定
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "As123459362",
    "db": "pttcrawler",
    "charset": "utf8",
    "autocommit":True
}
time_start = time.time() #開始計時

try:
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
      #資料表相關操作
        get_max_log_id="select MAX(UID) FROM log"
        # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
        command = "INSERT INTO thzu_data(log_UID, page, title, board, date, url)VALUES(%s, %s, %s, %s, %s, %s)"
        command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
        # 紀錄開始
        cursor.execute(command_log, (datetime.datetime.now(), "thzucrawler", "start", "", ""))
        # 取得 log 的 UID
        cursor.execute(get_max_log_id)
        log_UID = str(cursor.fetchone()[0])
        # 執行
        for raw_data in raw_datas:
            try:
                cursor.execute(command, (int(log_UID),int(raw_data["page"]), raw_data["title"], raw_data["board"], raw_data["date"], raw_data["url"]))
            except Exception as err:
                cursor.execute(command_log, (datetime.datetime.now(), "thzucrawler", "wrong", str(raw_data), str(err)))
                continue
        # 紀錄結束
        cursor.execute(command_log, (datetime.datetime.now(), "thzucrawler", "finish", "", ""))
except Exception as ex:
    print(ex)
        
time_end = time.time()    #結束計時
time_c= time_end - time_start   #執行所花時間
print('time cost', time_c, 's')

time cost 161.1300563812256 s
